In [1]:
###IMPORT MODULES
from re import A
import sys
import os
sys.path.append(os.path.abspath('../..'))

import numpy as np 
from tools.component_tools import Component
from tools.component_tools import Fluid
from tools.component_tools import Membrane
import tools.materials as materials
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from tools.component_tools import BreedingBlanket
import mph 
from tools.component_tools import Geometry
import math 
from tools import correlations as corr
inv_v=[]
inv_v_comsol=[]


: 

In [ ]:

for D in [1E-11]:
        U0=10
        eff_coms_v=np.array([])
        eff_py=np.array([])
        client = mph.start()
        compute_comsol=True
        # U0=1.5
        T=800
        c0=1E-2
        mat=materials.Flibe(T)
        T_hot_prim=900
        T_cold_prim=800
        Q=1E9
        d_hyd=1E-2
        L=20
        # D=1E-4
        thick=2E-3
        K_S=1E-2
        k_d=1E6
        k_r=1E6
        K_H=1E-1
        flibe=Fluid(T=T, Solubility=K_H, MS=False,D=mat.D*1E1, d_Hyd=d_hyd ,mu=mat.mu,rho=mat.rho,U0=U0,k=mat.k,
                cp=mat.cp)
        var_vec=[T,c0,d_hyd,L,D,thick,K_S,K_H,mat.D*1E1,U0]
        var_vec_str=['T','c0','d_hyd','L','D_St','thick','K_S','K_H','D_flibe','U0']
        Steel = Membrane( T=T,
                D=D,
                thick=thick,
                K_S=K_S,
                k_d=k_d,
                k_r=k_r,k=21)
        geom_pav=Geometry(L=L, D=d_hyd,thick=thick )
        PAV = Component(c_in=c0,  fluid=flibe, membrane=Steel,geometry=geom_pav)
        PAV.get_solid_inventory()
        PAV.use_analytical_efficiency()
        print("Efficiency "+str(PAV.eff))
        if compute_comsol:
                client = mph.start()
                if PAV.fluid.MS: 
                        
                        model = client.load('PAV_MS_2species_SST_copia.mph')  
                else: 
                        model = client.load('PAV_LM_SST.mph')
                for i,var in enumerate(var_vec):
                        model.parameter(var_vec_str[i], str(var))
                model.mesh()
                a=model.parameters()
                # model.save()

                model.solve()
                # Evaluate the results
                eff_coms = model.evaluate(['r', 'z', 'inv'], 'Study 1//Solution 1')

                # Remove NaN values from the 'inv' component
                inv_values = eff_coms[-1]  # Assuming 'inv' is the last component
                inv_values = inv_values[~np.isnan(inv_values)]

                # Calculate the average of the cleaned 'inv' values
                eff_coms_avg = np.average(inv_values)
                eff_coms_v = np.append(eff_coms_v, eff_coms_avg)
        else: 
                eff_coms_avg=4.22E-6
                # Append the average to the eff_coms_v array


        # Print the results
        print("PAV", (PAV.membrane.inv), "Coms=", eff_coms_avg, "%", "diff=", PAV.membrane.inv - eff_coms_avg, "%", "rel_diff=", (abs(PAV.membrane.inv - eff_coms_avg) / PAV.membrane.inv) * 100, "%")
        # if PAV.fluid.MS and compute_comsol:
        if compute_comsol:
                k_t_coms=model.evaluate(['r', 'z', 'k_t'], 'Study 1//Solution 1')
                k_t_values=k_t_coms[-1]
                k_t_values=k_t_values[~np.isnan(k_t_values)]
                k_t_avg=np.average(k_t_values)
                print("Trioma k_t=",PAV.fluid.k_t,"Comsol k_t=",k_t_avg)
                PAV.fluid.k_t=k_t_avg
                PAV.get_solid_inventory()
                print("PAV", (PAV.membrane.inv), "Coms=", eff_coms_avg, "%", "diff=", PAV.membrane.inv - eff_coms_avg, "%", "rel_diff=", (abs(PAV.membrane.inv - eff_coms_avg) / PAV.membrane.inv) * 100, "%")

                # Append the PAV membrane inventory to the eff_py array
        inv_v.append(PAV.membrane.inv)
        inv_v_comsol.append(eff_coms_avg)

w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833106.9828132356
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.3183854015
w is 1833107.318

In [3]:

from sympy import comp


print("ratio", PAV.membrane.inv/eff_coms_avg)

# Remove the model from the client
if compute_comsol:
    client.remove(model)


ratio 0.9997381796743289
